In [ ]:
!pip install torch==1.8.0+cu111 torchvision==0.9.0+cu111 torchaudio==0.8.0 -f https://download.pytorch.org/whl/torch_stable.html
import torch
print(torch.__version__)
print(torch.version.cuda)
!nvcc --version

Looking in links: https://download.pytorch.org/whl/torch_stable.html
1.8.0+cu111
11.1
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
%load_ext autoreload
%autoreload 2 

In [1]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
%cd /content/gdrive/MyDrive/Colab\ Notebooks/LidarObjectDetection/VoxelNet

/content/gdrive/MyDrive/Colab Notebooks/LidarObjectDetection/VoxelNet


In [ ]:
!pip install cloud-tpu-client==0.10 -q
!pip install https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.12.1-cp37-cp37m-linux_x86_64.whl -q

     |████████████████████████████████| 61kB 7.0MB/s 
ERROR: earthengine-api 0.1.269 has requirement google-api-python-client<2,>=1.12.1, but you'll have google-api-python-client 1.8.0 which is incompatible.
  ERROR: HTTP error 404 while getting https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.12.1-cp37-cp37m-linux_x86_64.whl
  ERROR: Could not install requirement torch-xla==1.12.1 from https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.12.1-cp37-cp37m-linux_x86_64.whl because of error 404 Client Error: Not Found for url: https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.12.1-cp37-cp37m-linux_x86_64.whl
ERROR: Could not install requirement torch-xla==1.12.1 from https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.12.1-cp37-cp37m-linux_x86_64.whl because of HTTP error 404 Client Error: Not Found for url: https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.12.1-cp37-cp37m-linux_x86_64.whl for URL https://storage.googleapis.com/tpu-p

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
%load_ext autoreload
%autoreload 2 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
%cd /content/gdrive/MyDrive/Colab\ Notebooks/LidarObjectDetection/VoxelNet

import torch
from torch import nn, optim
from torch.autograd import Variable
from torch.utils import data

import time
from model import VoxelNet
from loss import VoxelNetLoss
from config import config as cfg
from data.dataset import KittiDataset
import pathlib, os
import numpy as np

def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_uniform_(m.weight.data)
        m.bias.data.zero_()

# model
net = VoxelNet()
# try:
#   import torch_xla
#   import torch_xla.core.xla_model as xm
#   device = xm.xla_device()
# except: 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")

print("\n## Hardware type: {}\n".format(device.type))
net.to(device)

# training mode
net.train()

# initialization
print('Initializing weights...')
net.apply(weights_init)

# define optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# loss function
loss = VoxelNetLoss(alpha=1.5, beta=1)

def detection_collate(batch):
  voxel_features = []
  voxel_coords = []
  pos_equal_one = []
  neg_equal_one = []
  targets = []
  images = []
  calibs = []
  ids = []

  for i, sample in enumerate(batch):
    voxel_features.append(sample['voxel_features'])
    voxel_coords.append(
        np.pad(sample['voxel_coords'], ((0,0), (1,0)), mode='constant',
               constant_values=i
        )
    )
    pos_equal_one.append(sample['pos_equal_one'])
    neg_equal_one.append(sample['neg_equal_one'])
    targets.append(sample['target'])
    images.append(sample['rgb'])
    calibs.append(sample['calib'])
    ids.append(sample['file_id'])

    return np.concatenate(voxel_features), np.concatenate(voxel_coords), \
           np.array(pos_equal_one), np.array(neg_equal_one), \
           np.array(targets), images, calibs, ids

# dataset
print(pathlib.Path(os.getcwd()).absolute()/"data/training")
dataset=KittiDataset(pathlib.Path(os.getcwd()).absolute()/"data/training", cfg)
data_loader = data.DataLoader(dataset, batch_size=cfg.N, shuffle=True, 
                              num_workers=2, collate_fn=detection_collate,
                              pin_memory=False)
# training process
batch_iterator = None
epoch_size = len(dataset) // cfg.N
print('Epoch size', epoch_size)

for iteration in range(1):
  if (not batch_iterator) or (iteration % epoch_size == 0):
    batch_iterator = iter(data_loader)

  voxel_features, voxel_coords, pos_equal_one, neg_equal_one, targets, images, calibs, ids= next(batch_iterator)
  
  voxel_features = Variable(torch.cuda.FloatTensor(voxel_features))
  pos_equal_one = Variable(torch.cuda.FloatTensor(pos_equal_one))
  neg_equal_one = Variable(torch.cuda.FloatTensor(neg_equal_one))
  targets = Variable(torch.cuda.FloatTensor(targets))

  optimizer.zero_grad()

  t0 = time.time()
  psm, rm = net(voxel_features, voxel_coords)


/content/gdrive/MyDrive/Colab Notebooks/LidarObjectDetection/VoxelNet

## Hardware type: cuda

Initializing weights...
/content/gdrive/My Drive/Colab Notebooks/LidarObjectDetection/VoxelNet/data/training
Epoch size 3769
vwfs:  torch.Size([1, 128, 10, 400, 352])
cml: 1
conv3d


RuntimeError: ignored

In [ ]:
!sudo rm -rf ~/.nv

In [ ]:
%cd /content/gdrive/MyDrive/Colab\ Notebooks/LidarObjectDetection/VoxelNet
!python -m setup build_ext --inplace

/content/gdrive/MyDrive/Colab Notebooks/LidarObjectDetection/VoxelNet
running build_ext
building 'box_overlaps' extension
creating build
creating build/temp.linux-x86_64-3.7
creating build/temp.linux-x86_64-3.7/third_party
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-OGiuun/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-OGiuun/python3.7-3.7.10=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -fPIC -I/usr/local/lib/python3.7/dist-packages/numpy/core/include -I/usr/include/python3.7m -c ./third_party/box_overlaps.c -o build/temp.linux-x86_64-3.7/./third_party/box_overlaps.o
In file included from /usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1822:0,
                 from /usr/local/lib/python3.7/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
      

In [ ]:
!pip install pathlib

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
torch.__version__

'1.9.0+cu102'

In [ ]:
#Uninstall the current CUDA version
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'nvidia-kernel-common-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-325-updates' for glob 'nvidia*'
Note, selecting 'nvidia-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-binary' for glob 'nvidia*'
Note, selecting 'nvidia-331-dev' for glob 'nvidia*'
Note, selecting 'nvidia-304-updates-dev' for glob 'nvidia*'
Note, selecting 'nvidia-compute-utils-418-server' for glob 'nvidia*'
Note, selecting 'nvidia-384-dev' for glob 'nvidia*'
Note, selecting 'nvidia-libopencl1-346-updates' for glob 'nvidia*'
Note, selecting 'nvidia-driver-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-340-updates-uvm' for glob 'nvidia*'
Note, selecting 'nvidia-dkms-450-server' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-common' for glob 'nvidia*'
Note, selecting 'nvidia-kernel-source-440-server' for glob 'nvidia*'
Note, selecting 'nvidia-331-updates-uvm' for glob 'nvidi

In [ ]:
!sudo 31
apt --fix-broken install
#Download CUDA 10.2
!wget  --no-clobber https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-10-2_10.2.89-1_amd64.deb
#install CUDA kit dpkg
!dpkg -i cuda-10-2_10.2.89-1_amd64.deb
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!apt-get update
!apt-get install cuda-10-2
!nvcc --version

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Correcting dependencies... Done
The following additional packages will be installed:
  cuda-command-line-tools-10-2 cuda-compiler-10-2 cuda-cudart-10-2
  cuda-cudart-dev-10-2 cuda-cufft-10-2 cuda-cufft-dev-10-2 cuda-cuobjdump-10-2
  cuda-cupti-10-2 cuda-cupti-dev-10-2 cuda-curand-10-2 cuda-curand-dev-10-2
  cuda-cusolver-10-2 cuda-cusolver-dev-10-2 cuda-cusparse-10-2
  cuda-cusparse-dev-10-2 cuda-demo-suite-10-2 cuda-documentation-10-2
  cuda-driver-dev-10-2 cuda-drivers cuda-drivers-465 cuda-gdb-10-2
  cuda-libraries-10-2 cuda-libraries-dev-10-2 cuda-license-10-2
  cuda-memcheck-10-2 cuda-misc-headers-10-2 cuda-npp-10-2 cuda-npp-dev-10-2
  cuda-nsight-10-2 cuda-nsight-compute-10-2 cuda-nsight-systems-10-2
  cuda-nvcc-10-2 cuda-nvdisasm-10-2 cuda-nvgraph-10-2 cuda-nvgraph-dev-10-2
  cuda-nvjpeg-10-2 cuda-nvjpeg-dev-10-2 cuda-nvml-dev-10-2 cuda-nvprof-10-2
  cuda-nvprune-10-2 cuda-nvrtc-10-2 

In [ ]:
!apt --fix-broken install

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Correcting dependencies... Done
The following additional packages will be installed:
  cuda-command-line-tools-10-2 cuda-compiler-10-2 cuda-cudart-10-2
  cuda-cudart-dev-10-2 cuda-cufft-10-2 cuda-cufft-dev-10-2 cuda-cuobjdump-10-2
  cuda-cupti-10-2 cuda-cupti-dev-10-2 cuda-curand-10-2 cuda-curand-dev-10-2
  cuda-cusolver-10-2 cuda-cusolver-dev-10-2 cuda-cusparse-10-2
  cuda-cusparse-dev-10-2 cuda-demo-suite-10-2 cuda-documentation-10-2
  cuda-driver-dev-10-2 cuda-drivers cuda-drivers-465 cuda-gdb-10-2
  cuda-libraries-10-2 cuda-libraries-dev-10-2 cuda-license-10-2
  cuda-memcheck-10-2 cuda-misc-headers-10-2 cuda-npp-10-2 cuda-npp-dev-10-2
  cuda-nsight-10-2 cuda-nsight-compute-10-2 cuda-nsight-systems-10-2
  cuda-nvcc-10-2 cuda-nvdisasm-10-2 cuda-nvgraph-10-2 cuda-nvgraph-dev-10-2
  cuda-nvjpeg-10-2 cuda-nvjpeg-dev-10-2 cuda-nvml-dev-10-2 cuda-nvprof-10-2
  cuda-nvprune-10-2 cuda-nvrtc-10-2 

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Wed_Oct_23_19:24:38_PDT_2019
Cuda compilation tools, release 10.2, V10.2.89


In [5]:
%pycat .gitignore